In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, clear_output

In [2]:
dataset= pd.read_csv("stroke prediction.csv")
dataset.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0


In [3]:
dataset["gender"]=dataset["gender"].astype('category')
dataset["ever_married"]=dataset["ever_married"].astype('category')
dataset["Residence_type"]=dataset["Residence_type"].astype('category')
dataset["work_type"]=dataset["work_type"].astype('category')
dataset["smoking_status"]=dataset.smoking_status.astype('category')

In [5]:
dataset_imputed=dataset.copy()

In [6]:
X=dataset_imputed.drop(columns=["id","stroke"],axis=1)
y=dataset_imputed["stroke"]

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=88,stratify=y)

In [7]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()

X_train.gender=encoder.fit_transform(X_train.gender)

X_train.ever_married=encoder.fit_transform(X_train.ever_married)

X_train.work_type=encoder.fit_transform(X_train.work_type)

X_train.Residence_type=encoder.fit_transform(X_train.Residence_type)

X_train.smoking_status=X_train.smoking_status.map({'never smoked':0,'formerly smoked':1,'smokes':2})

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [8]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()

X_test.gender=encoder.fit_transform(X_test.gender)

X_test.ever_married=encoder.fit_transform(X_test.ever_married)

X_test.work_type=encoder.fit_transform(X_test.work_type)

X_test.Residence_type=encoder.fit_transform(X_test.Residence_type)

X_test.smoking_status=X_test.smoking_status.map({'never smoked':0,'formerly smoked':1,'smokes':2})

In [9]:
from sklearn.impute import KNNImputer

imputer=KNNImputer()

X_trimputed=pd.DataFrame(imputer.fit_transform(X_train),columns=X_train.columns)

X_teimputed=pd.DataFrame(imputer.transform(X_test),columns=X_test.columns)

In [10]:
X_trimputed["smoking_status"]=X_trimputed["smoking_status"].apply(round)
X_trimputed["smoking_status"].value_counts()

X_trimputed["smoking_status"]=X_trimputed["smoking_status"].astype('category')
X_trimputed["work_type"]=X_trimputed["work_type"].apply(int)
X_trimputed["work_type"]=X_trimputed["work_type"].astype('category')

In [11]:
X_teimputed["smoking_status"]=X_teimputed["smoking_status"].apply(round)
X_teimputed["smoking_status"].value_counts()

X_teimputed["smoking_status"]=X_teimputed["smoking_status"].astype('category')
X_teimputed["work_type"]=X_teimputed["work_type"].apply(int)
X_teimputed["work_type"]=X_teimputed["work_type"].astype('category')

In [12]:
X_trimputed=pd.concat([X_trimputed,pd.get_dummies(X_trimputed[["work_type","smoking_status"]]).drop(columns=["work_type_0","smoking_status_0"])],axis=1)
X_teimputed=pd.concat([X_teimputed,pd.get_dummies(X_teimputed[["work_type","smoking_status"]]).drop(columns=["work_type_0","smoking_status_0"])],axis=1)

In [13]:
X_trimputed.drop(columns=['work_type','smoking_status'],inplace=True)
X_teimputed.drop(columns=['work_type','smoking_status'],inplace=True)

In [14]:
X_trimputed

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,work_type_1,work_type_2,work_type_3,work_type_4,smoking_status_1,smoking_status_2
0,1.0,4.0,0.0,0.0,0.0,1.0,136.51,17.1,0,0,0,1,0,0
1,1.0,9.0,0.0,0.0,0.0,0.0,90.55,27.3,0,0,0,1,0,0
2,1.0,29.0,0.0,0.0,1.0,0.0,108.69,29.6,0,1,0,0,0,0
3,1.0,63.0,0.0,0.0,1.0,1.0,84.57,31.2,0,1,0,0,1,0
4,0.0,67.0,0.0,0.0,1.0,0.0,70.43,26.3,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34715,0.0,62.0,0.0,0.0,1.0,1.0,58.90,29.1,0,1,0,0,1,0
34716,0.0,43.0,0.0,0.0,0.0,1.0,94.82,29.7,0,0,0,0,0,0
34717,0.0,23.0,0.0,0.0,0.0,1.0,98.66,28.3,0,1,0,0,1,0
34718,1.0,9.0,0.0,0.0,0.0,1.0,150.00,17.4,0,0,0,1,0,0


In [16]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()

scaler.fit(X_trimputed)

X_trimputed=pd.DataFrame(scaler.transform(X_trimputed),columns=X_trimputed.columns)

X_teimputed=pd.DataFrame(scaler.transform(X_teimputed),columns=X_teimputed.columns)

In [ ]:
from sklearn.naive_bayes import B